# Sử dụng dataset có sẵn

In [1]:
import pandas as pd

# Đọc file song ngữ từ thư mục input
with open("/kaggle/input/assignment-nlp/en_sents.txt", encoding="utf-8") as f_en, \
     open("/kaggle/input/assignment-nlp/vi_sents.txt", encoding="utf-8") as f_vi:
    en_list = [line.strip() for line in f_en.readlines()]
    vi_list = [line.strip() for line in f_vi.readlines()]

assert len(en_list) == len(vi_list), "Số lượng câu không khớp!"

# Tạo DataFrame ban đầu
df_kaggle = pd.DataFrame({
    "en": en_list,
    "vi": vi_list
})

print(f"✅ Dataset từ Kaggle có {len(df_kaggle)} câu song ngữ.")
print(df_kaggle.tail())


✅ Dataset từ Kaggle có 254090 câu song ngữ.
                                            en  \
254085           What sound does a sheep make?   
254086  Tom stayed in Boston for three months.   
254087             Tom borrowed $300 from Mary   
254088              Why aren't you interested?   
254089                  You were at my wedding   

                                      vi  
254085   một con cừu tạo ra âm thanh gì?  
254086  Tom ở lại boston trong ba tháng.  
254087          tom đã vay $ 300 từ mary  
254088       tại sao bạn không quan tâm?  
254089         bạn đã ở đám cưới của tôi  


# Tải dataset từ corpora

In [2]:
import os
import requests, zipfile, io

corpora = {
    "TED2020": "https://object.pouta.csc.fi/OPUS-TED2020/v1/moses/en-vi.txt.zip",
    "WikiMatrix": "https://object.pouta.csc.fi/OPUS-WikiMatrix/v1/moses/en-vi.txt.zip",
}

os.makedirs("opus_en_vi", exist_ok=True)
all_en, all_vi = [], []

for name, url in corpora.items():
    print(f"📥 Đang tải: {name}...")
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    extract_path = os.path.join("opus_en_vi", name)
    os.makedirs(extract_path, exist_ok=True)
    z.extractall(extract_path)
    
    # Tên file trong mỗi thư mục theo format: {CorpusName}.en-vi.en / .vi
    en_path = os.path.join(extract_path, f"{name}.en-vi.en")
    vi_path = os.path.join(extract_path, f"{name}.en-vi.vi")

    if os.path.exists(en_path) and os.path.exists(vi_path):
        with open(en_path, encoding='utf-8') as f_en, open(vi_path, encoding='utf-8') as f_vi:
            en_lines = f_en.readlines()
            vi_lines = f_vi.readlines()
            
            if len(en_lines) == len(vi_lines):
                all_en.extend([line.strip() for line in en_lines])
                all_vi.extend([line.strip() for line in vi_lines])
            else:
                print(f"⚠️ Warning: {name} có số dòng không khớp.")

print(f"✅ Đã tải thêm {len(all_en)} cặp câu.")


📥 Đang tải: TED2020...
📥 Đang tải: WikiMatrix...
✅ Đã tải thêm 1400169 cặp câu.


In [3]:
df_new = pd.DataFrame({
    "en": all_en,
    "vi": all_vi
})

print(df_new.tail())

                                                        en  \
1400164  Conservation bills passed by the government in...   
1400165  Farragut's group came under heavy air attack t...   
1400166  The first compilation volume of Sugimoto's wor...   
1400167  Starting in the mid-1990s, the Strip became a ...   
1400168                                                      

                                                        vi  
1400164  Các dự luật bảo tồn được chính phủ thông qua b...  
1400165  Đội của nó chịu đựng một đợt không kích ác liệ...  
1400166  Tập tankōbon đầu tiên của chuyển thể này phát ...  
1400167  Bắt đầu trong giữa thập niên 1990, dải đã trở ...  
1400168                                                     


# Sử dụng Gemini 2.0 Flash để genarate thêm các cặp song ngữ

In [ ]:
import time
import json
import pandas as pd
from google.generativeai import configure, GenerativeModel
from tqdm import tqdm

prompt = """
    Hãy tạo ra 50 cặp câu song ngữ tiếng Anh và tiếng Việt. 
    Mỗi cặp chỉ gồm một câu ngắn (10 đến 20 từ), về các chủ đề giao tiếp hằng ngày. 
    Trả về dưới dạng văn bản, mỗi cặp câu trên một dòng như sau (chỉ cần cặp câu,
    không cần đánh số trước mỗi câu):
    Anh: "I'm going to school now." - Việt: "Tôi đang đi học bây giờ."
    Anh: "This is a nice day." - Việt: "Hôm nay là một ngày đẹp."
    ...
    """

configure(api_key=API_KEY)  # ← THAY API KEY TẠI ĐÂY

model = GenerativeModel("gemini-2.0-flash")

all_pairs = []

# Lặp lại 400 lần để sinh tổng cộng 20,000 cặp câu
for i in tqdm(range(400), desc="🔄 Generating batches"):
    try:
        response = model.generate_content(prompt)
        content = response.text.strip()

        lines = content.split("\n")

        for line in lines:
            try:
                if "Anh:" in line and "Việt:" in line:
                    en_vi_pair = line.split(" - ")
                    if len(en_vi_pair) == 2:
                        en_sentence = en_vi_pair[0].replace('Anh: "', '').replace('"', '').strip()
                        vi_sentence = en_vi_pair[1].replace('Việt: "', '').replace('"', '').strip()
                        all_pairs.append({"en": en_sentence, "vi": vi_sentence})
            except Exception as e:
                tqdm.write(f"⚠️ Error in line parsing: {line} - {e}")
    except Exception as e:
        tqdm.write(f"⚠️ Error at batch {i+1}: {e}")

    time.sleep(1)  # Tránh spam API


🔄 Generating batches: 100%|██████████| 400/400 [41:34<00:00,  6.24s/it]


In [5]:
df_gen = pd.DataFrame(all_pairs)
print(df_gen.tail())

                     en               vi
20187        Take care.       Bảo trọng.
20188       I miss you.     Tôi nhớ bạn.
20189       I love you.     Tôi yêu bạn.
20190  Congratulations!       Chúc mừng!
20191   That's amazing!  Thật tuyệt vời!


In [6]:
def show_sample_df(df_parallel):
    print(f"──────────────────────────")
    # Hiển thị 5 mẫu ngẫu nhiên để kiểm tra
    random_samples = df_parallel.sample(5, random_state=42)  # random_state để kết quả có thể reproduce
    print("🎲 5 mẫu ngẫu nhiên từ dữ liệu sau tiền xử lý:\n")
    
    for idx, row in random_samples.iterrows():
        print(f"EN: {row['en']}")
        print(f"VI: {row['vi']}")
        print(f"──────────────────────────")

def save_to_csv(df, file_path):
    """
    Lưu DataFrame thành file CSV
    
    Parameters:
        df: DataFrame cần lưu
        file_path: Đường dẫn file output (vd: 'dataset.csv')
    """
    df.to_csv(file_path, index=False, encoding='utf-8')
    print(f"✅ Đã lưu file: {file_path}")

# Nối 3 DataFrame theo chiều dọc (axis=0)
df_parallel = pd.concat([df_gen, df_new, df_kaggle], axis=0, ignore_index=True)

# Kiểm tra kết quả
print("Shape của df_gen:", df_gen.shape)
print("Shape của df_new:", df_new.shape)
print("Shape của df_kaggle:", df_kaggle.shape)
print("\nShape sau khi nối:", df_parallel.shape)
show_sample_df(df_parallel)
save_to_csv(df_parallel,"dataset.csv")


Shape của df_gen: (20192, 2)
Shape của df_new: (1400169, 2)
Shape của df_kaggle: (254090, 2)

Shape sau khi nối: (1674451, 2)
──────────────────────────
🎲 5 mẫu ngẫu nhiên từ dữ liệu sau tiền xử lý:

EN: You're welcome.
VI: Không có gì.
──────────────────────────
EN: Large mines were also present across the Red Sea in what is now Saudi Arabia.
VI: Những mỏ lớn cũng hiện diện trên khắp Biển Đỏ ở nơi hiện là Ả Rập Xê Út.
──────────────────────────
EN: Tom knew that Mary was rich.
VI: Tom biết rằng mary rất giàu.
──────────────────────────
EN: Attacks on the island of Ceylon were also carried out.
VI: Các cuộc tấn công lên đảo Ceylon cũng được thực hiện.
──────────────────────────
EN: We tell the whole story, we explain who the people are, how we feel about them; we spell out all the inside jokes.
VI: Chúng ta phải giải thích toàn bộ, chúng ta là ai, thấy họ thế nào; chúng ta giải thích cặn kẽ những câu đùa ít người biết.
──────────────────────────
✅ Đã lưu file: dataset.csv


# Tiền xử lý văn bản

In [7]:
import pandas as pd
import numpy as np
from multiprocessing import Pool

# Hàm xử lý song song
def parallel_preprocess(text):
    text = str(text).strip()  # Loại bỏ khoảng trắng thừa
    text = ' '.join(text.split())  # Chuẩn hóa khoảng trắng
    return text

# Áp dụng song song cho cả 2 cột
def preprocess_df(df):
    with Pool() as pool:
        df['en'] = pool.map(parallel_preprocess, df['en'])
        df['vi'] = pool.map(parallel_preprocess, df['vi'])
    return df

# Áp dụng tiền xử lý (chạy 1 lần)
df = preprocess_df(df_parallel)


In [8]:
# Lọc câu trống hoặc NaN
df = df[
    df['en'].notna() & 
    df['vi'].notna() &
    (df['en'] != '') & 
    (df['vi'] != '')
]

# Lọc câu quá ngắn/dài (tối ưu bằng vectorization)
df['en_len'] = df['en'].str.split().str.len()
df['vi_len'] = df['vi'].str.split().str.len()

df = df[
    (df['en_len'] >= 5) & (df['en_len'] <= 100) &
    (df['vi_len'] >= 5) & (df['vi_len'] <= 100)
].drop(columns=['en_len', 'vi_len'])

print(len(df))


<ipython-input-8-891d4e71bbe9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['en_len'] = df['en'].str.split().str.len()
<ipython-input-8-891d4e71bbe9>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vi_len'] = df['vi'].str.split().str.len()


1551795


In [9]:
# Loại bỏ trùng lặp chính xác
df = df.drop_duplicates()

# Loại bỏ trùng lặp gần đúng (dùng hash cho nhanh)
df['hash'] = df['en'] + '|||' + df['vi']
df = df.drop_duplicates('hash').drop(columns=['hash'])
print(len(df))

1549097


In [10]:
import re

# Biên dịch regex trước để tăng tốc
noise_pattern = re.compile(r'[^\w\s,.?!]|http\S+|www\S+')

def filter_noise(text):
    return bool(noise_pattern.search(text))

# Lọc câu chứa ký tự nhiễu (áp dụng song song)
with Pool() as pool:
    en_has_noise = pool.map(filter_noise, df['en'])
    vi_has_noise = pool.map(filter_noise, df['vi'])

df = df[~(np.array(en_has_noise) | np.array(vi_has_noise))]
print(len(df))

746390


**Sau khi tiền xử lý ta có dataset sau**

In [11]:
# Lưu cột tiếng Anh
df['en'].to_csv("en_sents_dataset.txt", index=False, header=False)

# Lưu cột tiếng Việt
df['vi'].to_csv("vi_sents_dataset.txt", index=False, header=False)

print("Đã lưu thành công hai file: en_sents_dataset.txt và vi_sents_dataset.txt")


Đã lưu thành công hai file: en_sents_dataset.txt và vi_sents_dataset.txt
